In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import pandas as pd
import csv

In [5]:
url = 'https://www.vivino.com/explore?e=eJwdijsOgCAQBW_zaqDfzo4jGGPWFQmJgAH83V5iM1PMxEIaMSRSiPyQ0UpBXhospMPi6NlvdHEJrvGOvFDhFpKvs-QzNWRaXRXcbZz6-st8itQcHw%3D%3D'
product_names = []
profile = webdriver.FirefoxProfile()
profile.set_preference("general.useragent.override", "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:81.0) Gecko/20100101 Firefox/81.0")
options = Options()
options.headless = True
browser = webdriver.Firefox(options=options)
options.add_argument("start-maximized")
options.add_argument('disable-infobars')
browser.get(url)
titles = browser.find_elements_by_class_name('vintageTitle__wine--U7t9G')
ratings = browser.find_elements_by_class_name('vivinoRating__averageValue--3Navj')
    
with open('vinData.csv', 'a', encoding='UTF8') as f:
    writer = csv.writer(f)
for title, rating in zip(titles, ratings):
    data = [title.text, rating.text]
    writer.writerow(data)
    
print(data)

ValueError: I/O operation on closed file.